In [13]:
import datetime  # For datetime objects
#import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
import pandas as pd
import time as tm


import matplotlib.pyplot as plt
import backtrader.analyzers as btanalyzers
import math
import numpy as np
import quantstats

# Import the backtrader platform
import backtrader as bt
import akshare as ak
import os
#%matplotlib auto
import telegram
TOKEN = '1454306275:AAHbxYSgcGBoIY3t5k0BYTuyOQEU5YYn1Wo'
bot = telegram.Bot(TOKEN)

from jqdatasdk import *
auth('15*********','********') 
count=get_query_count()
print(count)

def jq_data(code,time):
    code =normalize_code(code)
    data = get_bars(code,count=2000, unit=time,fields=['date','open','high','low','close'],include_now=False,end_dt=datetime.datetime.now(),fq_ref_date=datetime.datetime.now())
    data['date'] = pd.to_datetime(data['date'])
    data.set_index("date",inplace=True)
    tm.sleep(1)
    return data 
    
def get_data(code,time):
            data= ak.futures_zh_minute_sina(symbol=code, period=time)
            data['date'] = pd.to_datetime(data['date'])
            data.set_index("date",inplace=True)
            data.drop(columns=['hold'],inplace = True)
            data=data.astype({
                    'open': 'float',
                    'high':'float',
                    'low':'float',
                    'close':'float',
                    'volume':'float'
                })
            tm.sleep(1)
            return data 
def Tg_send(time,code,signal,close):
            f = open('/code/log.txt', 'r').read()
            x = "%s,code=%s,%s" % (time,code,signal)
            #print(f)
            #print(x)
            if x in f :
                print("当天已经执行过了")
            else:
                print("当前时间发出%s信号,正在推送TG..."% (signal))
                bot.send_message(chat_id='727256696', text="品种:%s ,价格:%.2f ,发出%s信号！" % (code,close,signal))
                os.system('echo %s,code=%s,%s >>/code/log.txt' % (time,code,signal) )
        
# Create a Stratey
class TestStrategy(bt.Strategy):
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('{}, {}'.format(dt.isoformat(), txt))
        
        
    def __init__(self):
        # 保存现有持仓的股票
        self.position_dict={}

        self.inds = dict()
        for d in self.datas:
            
            self.inds[d] = dict()
            # Add a MovingAverageSimple indicator
            # 50日移动平均线
            self.inds[d]['sma50'] = bt.indicators.SimpleMovingAverage(d.close,period=50)
            # 100日移动平均线
            self.inds[d]['sma100'] = bt.indicators.SimpleMovingAverage(d.close,period=100)
            # 100 日真实波动ATR
            self.inds[d]['atr100'] = bt.indicators.AverageTrueRange(d, period=100)
            # 50日的收盘最高价
            self.inds[d]['High50'] = bt.indicators.Highest(d.close(-1), period=50,subplot=False)
            # 50日的收盘最低价
            self.inds[d]['Low50'] = bt.indicators.Lowest(d.close(-1), period=50,subplot=False)

        

        
    def prenext(self):
        #print("prenext")
        self.next()


        
        

    def next(self):

        for d in self.datas:
            now=datetime.datetime.now()
            now_str=now.strftime('%Y-%m-%d')
            #now_str = "2020-12-24"
            dt_str = d.datetime.date(0).strftime('%Y-%m-%d')
            pos = self.getposition(d).size
            getcash = self.broker.getvalue()
            #print(pos)
            if  pos ==0:  # no market / no orders
                if self.inds[d]['sma50'][0] > self.inds[d]['sma100'][0] and d.close[0] > self.inds[d]['High50'][0]:
                        if d._name in self.position_dict:
                            #print("已经有做多的订单了")
                            continue

                        cash=(0.01*getcash)/(3*self.inds[d]['atr100'][0]/d.close[0])
                        #print(cash)
                        size = cash/(d.close[0]*10)
                        #print(size)
                        cerebro.broker.setcommission(commission=1.0,margin=int(d.close[0]),mult=10,name=d._name)
                        self.order = self.buy(data=d,size = size)
                        self.position_dict[d._name] = self.order
                        self.log('做多品种:' + d._name + ',做多价格： %.2f' % d.close[0])
                        
                        if dt_str == now_str:
                            Tg_send(time=now_str,code = d._name,signal = "做多",close = d.close[0])
            

                if self.inds[d]['sma50'][0]  < self.inds[d]['sma100'][0] and d.close[0] < self.inds[d]['Low50'][0]:
                    if d._name in self.position_dict:
                            #print("已经有做空的订单了")
                            continue
                    
                    cash=(0.01*getcash)/(3*self.inds[d]['atr100'][0]/d.close[0])
                    #print(cash)
                    size = cash/(d.close[0]*10)
                    #print(size)
                    cerebro.broker.setcommission(commission=1.0,margin=int(d.close[0]),mult=10,name=d._name)
                    self.order = self.sell(data=d,size = size)
                    self.position_dict[d._name] = self.order
                    
                    self.log('做空品种:' + d._name + ',做空价格： %.2f' % d.close[0])
                    if dt_str == now_str:
                        Tg_send(time=now_str,code = d._name,signal = "做空",close = d.close[0])

            else :
                if self.getposition(d).size > 0  and d.close[0] < self.inds[d]['High50'][0] - 3*self.inds[d]['atr100'][0]:
                        if d._name in self.position_dict:
                            self.order = self.close(data=d)
                            #print(self.getposition(d).size)
                            self.log('平多品种:' + d._name + ',平多价格： %.2f' % d.close[0])
                            self.position_dict.pop(d._name)
                            if dt_str == now_str:
                                Tg_send(time=now_str,code = d._name,signal = "平多",close = d.close[0])
                            
                
                if self.getposition(d).size < 0 and d.close[0] > self.inds[d]['Low50'][0] + 3*self.inds[d]['atr100'][0]:
                        if d._name in self.position_dict:
                            self.order = self.close(data=d)
                            self.log('平空品种:' + d._name + ',平空价格： %.2f' % d.close[0])
                            self.position_dict.pop(d._name)

                            if dt_str == now_str:
                                Tg_send(time=now_str,code = d._name,signal = "平空",close = d.close[0])

    def notify_order(self, order):
        
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        if order.status == order.Rejected:
            self.log(f"Rejected : order_ref:{order.ref}  data_name:{order.p.data._name}")
            
        if order.status == order.Margin:
            self.log(f"Margin : order_ref:{order.ref}  data_name:{order.p.data._name}")
            
        if order.status == order.Cancelled:
            self.log(f"Concelled : order_ref:{order.ref}  data_name:{order.p.data._name}")
            
        if order.status == order.Partial:
            self.log(f"Partial : order_ref:{order.ref}  data_name:{order.p.data._name}")
         
        if order.status == order.Completed:
            if order.isbuy():
                self.log(f" BUY : data_name:{order.p.data._name} price : {order.executed.price} , cost : {order.executed.value} , commission : {order.executed.comm}")

            else:  # Sell
                self.log(f" SELL : data_name:{order.p.data._name} price : {order.executed.price} , cost : {order.executed.value} , commission : {order.executed.comm}")
    
    def notify_trade(self, trade):
        # 一个trade结束的时候输出信息
        if trade.isclosed:
            self.log('closed symbol is : {} , total_profit : {} , net_profit : {}' .format(
                            trade.getdataname(),trade.pnl, trade.pnlcomm))
            # self.trade_list.append([self.datas[0].datetime.date(0),trade.getdataname(),trade.pnl,trade.pnlcomm])
            
        if trade.isopen:
            self.log('open symbol is : {} , price : {} ' .format(
                            trade.getdataname(),trade.price))

    def stop(self):
        self.log('结束')


if __name__ == '__main__':

    # 初始化模型
    cerebro = bt.Cerebro()

    # 构建策略
    strats = cerebro.addstrategy(TestStrategy)


#     #品种池
    code_range =['UR0','JD0','C0','M0','RM0','TA0','MA0','FG0','PF0','SA0','BU0','FU0',
                 'CU0','AL0','ZN0','RB0','RU0','HC0','SS0','P0','J0','Y0','JM0','I0','AP0','CJ0',
                 'SR0','CF0','T0']
    #code_range = ['C0','M0','A0','Y0','OI0','P0','CF0','SR0','TA0','AU0','CU0','AL0','ZN0','RB0','RU0','L0','V0']
#     code_range =['SP2106','JD2109','PF2109','C2109','M2109','RM2109','TA2109','MA2109','FG2109','SA2109','BU2109','FU2109','AG2106',
#                         'RB2110','L2109','PP2109','SF2109','AP2110','CJ2109','PK2110','SR2109','P2109']
    #code_range =['SP2106','PF2105','C2105','M2105','AP2110','CJ2109','PK2110','P2109','FU2109','RB2110','L2109','PP2109','SF2109']
    #code_range = ['C2105']
    code_range =['JD9999','BU9999','UR9999','C9999','M9999','RM9999','TA9999','MA9999','FG9999','PF9999','SA9999','FU9999',
                 'CU9999','AL9999','ZN9999','RB9999','RU9999','HC9999','SS9999','P9999','J9999','Y9999','JM9999','I9999','AP9999','CJ9999',
                 'SR9999','CF9999','T9999']
    #code_range =['C']
    #qh_c= ak.stock_zh_a_daily(symbol="sh600000", adjust="hfq")
    #start_date = datetime.datetime(2021,3,8)  # 回测开始时间
    #end_date = datetime.datetime(2021,4,30)  # 回测结束时间
    #data = bt.feeds.PandasData(dataname=qh_c, fromdate=start_date, todate=end_date)  # 加载数据
    for code_name in code_range:
        #dataname = get_data(code_name,"60")
        dataname = jq_data(code_name,'60m')  #'1m', '5m', '15m', '30m', '60m', '120m', '1d', '1w'(一周), '1M'（一月）
        print(code_name)
        print(dataname)
        data = bt.feeds.PandasData(dataname=dataname,timeframe=bt.TimeFrame.Minutes,compression=60)  # 加载数据
        cerebro.adddata(data,name = code_name)  # 将数据传入回测系统
    
    # 设定初始资金和佣金
    cerebro.broker.setcash(10000000.0)
    #cerebro.broker.setcommission(0.0002)
    #cerebro.addsizer(bt.sizers.PercentSizer, percents=12)

    # 以发出信号当日收盘价成交
    #cerebro.broker.set_coc(True)
    # 策略执行前的资金
    print('启动资金: %.2f' % cerebro.broker.getvalue())

    # 策略执行
    #cerebro.run()
    print('结束资金: %.2f' % cerebro.broker.getvalue())
    

    cerebro.addanalyzer(btanalyzers.SharpeRatio, _name = 'sharpe')
    cerebro.addanalyzer(btanalyzers.DrawDown, _name = 'drawdown')
    cerebro.addanalyzer(btanalyzers.Returns, _name='returns')
    cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='track')
    cerebro.addanalyzer(bt.analyzers.PyFolio, _name='PyFolio')



    cerebro.broker.set_fundstartval(1)
    print(f'Starte Portfolio Value {cerebro.broker.getvalue()}')
    result = cerebro.run()
    print('----------------------------')
    print(f'End portfolio value {cerebro.broker.getvalue()}')
    print(cerebro.broker.get_fundvalue())
    print('----------------------------')
    print(f"Total Return:  {round(result[0].analyzers.returns.get_analysis()['rtot']*100, 2)}%")
    print(f"APR:           {round(result[0].analyzers.returns.get_analysis()['rnorm100'],2)}%")
    print(f"Max DrawDown:  {round(result[0].analyzers.drawdown.get_analysis()['max']['drawdown'],2)}%")
    #print(f"Sharpe Ratio:  {round(result[0].analyzers.sharpe.get_analysis()['sharperatio'],2)}")
    print(result[0].analyzers.track.get_analysis())
    portfolio_stats = result[0].analyzers.getbyname('PyFolio')
    returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
    returns.index = returns.index.tz_convert(None)
    transactions.index = transactions.index.tz_convert(None)
    quantstats.reports.html(returns,output=f'/data/muma/index.html', title=f' 期货多品种交易系统 Analysis')
  
    #cerebro.plot(iplot=True,style = "bar",barup = "red",bardown ="green")  # 绘图


{'total': 1000000, 'spare': 1000000}
JD9999
                       open    high     low   close
date                                               
2019-08-28 10:00:00  4341.0  4437.0  4341.0  4417.0
2019-08-28 11:00:00  4417.0  4448.0  4408.0  4444.0
2019-08-28 11:30:00  4443.0  4459.0  4433.0  4459.0
2019-08-28 14:00:00  4459.0  4462.0  4369.0  4383.0
2019-08-28 15:00:00  4383.0  4393.0  4357.0  4368.0
...                     ...     ...     ...     ...
2021-04-22 10:00:00  4990.0  5029.0  4922.0  4965.0
2021-04-22 11:00:00  4965.0  4993.0  4942.0  4984.0
2021-04-22 11:30:00  4983.0  5049.0  4980.0  5023.0
2021-04-22 14:00:00  5021.0  5023.0  4980.0  4999.0
2021-04-22 15:00:00  4998.0  5001.0  4964.0  4979.0

[2000 rows x 4 columns]
BU9999
                       open    high     low   close
date                                               
2020-01-16 11:30:00  3172.0  3176.0  3170.0  3176.0
2020-01-16 14:00:00  3174.0  3180.0  3170.0  3174.0
2020-01-16 15:00:00  3176.0  3180.0  316